In [ ]:
%load_ext autoreload
%autoreload 2

from gnet_model import GNLightning
import torch

torch.cuda.empty_cache()

model = GNLightning(
    d_model=128,
    lr=1e-3,
    batch_size=32,
    num_workers=8,
    train_path=r'path to the output of ptn2gcn with split=train',
    val_path=r'path to the output of ptn2gcn with split=val')

device = torch.device('cuda:0')
model.to(device)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


GNLightning(
  (gnet): GraphNetwork(
    (conv1): GCNConv(8, 128)
    (conv2): GCNConv(128, 128)
    (lin1): Linear(in_features=256, out_features=128, bias=True)
    (lin_final): Linear(in_features=128, out_features=3, bias=True)
  )
  (criterion): NLLLoss()
)

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import Trainer

# logger = WandbLogger(project='parallel_tables', name='gnet_attempt_1')

# Set up checkpointing and trainer
checkpoint_callback = ModelCheckpoint(
    monitor="validation_loss",
    filename="detr-{epoch:02d}-{validation_loss:.2f}",
    save_top_k=3,
    mode="min",
    save_last=True,
    dirpath=r'..\checkpoints'
)

trainer = Trainer(
    max_epochs=50,
    log_every_n_steps=5,
    # logger=logger,
    callbacks=[checkpoint_callback],
    val_check_interval=0.3,  # Run validation 2 times per epoch
    devices=1,
    accelerator="gpu")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [7]:
trainer.fit(model)

c:\Users\remote desktop\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_lightning\loggers\wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
c:\Users\remote desktop\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:652: Checkpoint directory C:\Users\remote desktop\Downloads\DETR-GFTE\checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type         | Params | Mode 
---------------------------------------------------
0 | gnet      | GraphNetwork | 50.9 K | train
1 | criterion | NLLLoss      | 0      | train
---------------------------------------------------
50.9 K    Trainable params
0         Non-trainable params
50.9 K    Total params
0.204     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\remote desktop\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:419: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


c:\Users\remote desktop\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:419: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Epoch 0:  30%|██▉       | 4676/15587 [8:56:54<20:52:49,  0.15it/s, v_num=8dg0]Error: Expected input batch_size (34480) to match target batch_size (30666).
Error: Expected input batch_size (27002) to match target batch_size (24390).
Epoch 0:  60%|█████▉    | 9352/15587 [18:27:59<12:18:42,  0.14it/s, v_num=8dg0]Error: Expected input batch_size (34480) to match target batch_size (30666).
Error: Expected input batch_size (27002) to match target batch_size (24390).
Epoch 0:  90%|████████▉ | 14028/15587 [27:59:21<3:06:38,  0.14it/s, v_num=8dg0]Error: Expected input batch_size (34480) to match target batch_size (30666).
Error: Expected input batch_size (27002) to match target batch_size (24390).
Epoch 1:  18%|█▊        | 2866/15587 [5:30:04<24:25:02,  0.14it/s, v_num=8dg0]

c:\Users\remote desktop\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_lightning\trainer\call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
